In [2]:
import pandas as pd 
from pandas import DataFrame as df
from bs4 import BeautifulSoup
import io
import requests
from datetime import datetime as dt

# Web Scraping

In [3]:

url = f'https://www.set.or.th/set/factsheet.do?symbol=jts&ssoPageId=3&language=th&country=TH'

source = requests.get(url)
soup = BeautifulSoup(source.content, 'lxml', from_encoding='utf-8')
table = soup.find('table', class_='table-factsheet-padding3')
rows = table.find_all('tr')
table.find('td', class_='factsheet-heading2').text


'บริษัท จัสมิน เทคโนโลยี โซลูชั่น จำกัด (มหาชน)'

In [4]:
ListedCom = pd.read_excel('ListedCompanies.xlsx'
,converters = {'Symbols':str}
)

ListedCom.head()

,Symbols
0,2S
1,3K-BAT
2,7UP
3,A
4,A5


In [5]:
names = ListedCom['Symbols'].values.tolist()
data_all = []
next_ = None
for name in names :
  data_s = [name]
  url = 'https://www.set.or.th/set/factsheet.do?symbol='+name+'&ssoPageId=3&language=th&country=TH'
  source = requests.get(url)
  soup = BeautifulSoup(source.content, 'lxml', from_encoding='utf-8')
  tables = soup.find_all('table')
  text = []
  for table_t in tables :
      rows = table_t.find_all('tr')
      for row in rows :
          data = row.find_all('td')
          for element in data :
              element_t = element.text
              text += [element_t]
  columns = [name,'Market','Industry','URL','วันที่ก่อตั้งบริษัท','วันที่จดทะเบียนกับตลท.','ลักษณะธุรกิจ','นโยบายเงินปันผล','% /จำนวนผู้ถือหุ้นรายย่อย']   
  for column in columns :
      for index, obj in enumerate(text):
            if column == 'Market' and (obj[0:9] == 'CG Report') :
                data = text[index - 1]
            elif column == 'Industry' and (obj[0:9] == 'CG Report') :
                data = text[index - 2]
            elif column == '% /จำนวนผู้ถือหุ้นรายย่อย' and obj == column :
                data = text[index + 2]
            elif obj == column :
                data = text[index + 1]
      data_s +=[data]
  data_all +=[data_s]

In [6]:
ListStock = df(data_all,columns=['Symbol','FullName','Market','Industry','URL','วันที่ก่อตั้งบริษัท','วันที่จดทะเบียนกับตลท.','ลักษณะธุรกิจ','นโยบายเงินปันผล','% /จำนวนผู้ถือหุ้นรายย่อย'])

ListStock.head()

,Symbol,FullName,Market,Industry,URL,วันที่ก่อตั้งบริษัท,วันที่จดทะเบียนกับตลท.,ลักษณะธุรกิจ,นโยบายเงินปันผล,% /จำนวนผู้ถือหุ้นรายย่อย
0,2S,บริษัท 2 เอส เมทัล จำกัด (มหาชน),SET,สินค้าอุตสาหกรรม/เหล็ก และ ผลิตภัณฑ์โลหะ,www.ss.co.th,29/05/1992,02/09/2552,แปรรูปเหล็กรูปพรรณ และจัดหาเหล็กรูปพรรณเพื่อจำ...,ไม่ต่ำกว่าร้อยละ 40 ของกำไรสุทธิหลังหักภาษีเงิ...,58.11%
1,3K-BAT,บริษัท ไทย เอ็นเนอร์จี สโตเรจ เทคโนโลยี จำกัด ...,SET,สินค้าอุตสาหกรรม/ยานยนต์,-,01/10/2020,01/10/2563,ผลิตและจำหน่ายแบตเตอรี่หลากหลายรูปแบบ ได้แก่ 1...,ไม่เกินร้อยละ 50 ของกำไรสุทธิของงบการเงิน (โดย...,2.10%
2,7UP,บริษัท เซเว่น ยูทิลิตี้ส์ แอนด์ พาวเวอร์ จำกัด...,SET,ทรัพยากร/พลังงานและสาธารณูปโภค,www.sevenup.co.th,18/9/1995,03/04/2545,กลุ่มบริษัทประกอบธุรกิจหลัก 5 ธุรกิจ ได้แก่ 1)...,ไม่ต่ำกว่าร้อยละ 50 ของกำไรสุทธิที่เหลือหลังหั...,66.55%
3,A,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน),SET,อสังหาริมทรัพย์และก่อสร้าง/พัฒนาอสังหาริมทรัพย...,http://www.areeya.co.th,20/1/2000,01/04/2547,ธุรกิจอสังหาริมทรัพย์ แบ่งเป็น 3 ประเภท 1) โคร...,ประมาณร้อยละ 40 ของกำไรสุทธิหลังหักภาษีเงินได้...,25.86%
4,A5,บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน),mai,บริการ/,www.assetfive.co.th,19/09/2002,17/11/2546,เป็นบริษัทโฮลดิ้งคอมปานีเพื่อลงทุนในบริษัทพัฒน...,ไม่ต่ำกว่าร้อยละ 50 ของกำไรสุทธิหลังหักภาษีเงิ...,20.60%
